In [ ]:
print("Hello, Jupyter!")

## セットアップ

In [ ]:
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient
from dotenv import load_dotenv  # .env ファイルを読み込むためのライブラリ
import os
import traceback  # エラーの詳細を出力するためのモジュール

# .env ファイルを読み込む
load_dotenv()

# Azure AD テナント ID、クライアント ID、クライアントシークレット
tenant_id = os.getenv("TENANT_ID")
client_id = os.getenv("CLIENT_ID")
print(client_id)
client_secret = os.getenv("CLIENT_SECRET")
user_id = os.getenv("USER_ID")  # 取得したいユーザーの ID を指定
room_id = os.getenv("ROOM_ID")  # 取得したい会議室の ID を指定

# 必要なスコープ
scopes = ['https://graph.microsoft.com/.default']

try:
    # ClientSecretCredential を使用して認証
    credential = ClientSecretCredential(
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret
    )

    # Microsoft Graph クライアントを作成
    graph_client = GraphServiceClient(credential, scopes)
    print("Microsoft Graph クライアントの作成に成功しました。")
except Exception as e:
    print("Microsoft Graph クライアントの作成に失敗しました:")
    print({e})
    print("----------")
    traceback.print_exc()  # エラーの詳細を出力

## ユーザー情報の取得
https://learn.microsoft.com/ja-jp/graph/api/user-get?view=graph-rest-1.0&tabs=python#request

In [ ]:
import asyncio

# ユーザーを取得する非同期関数
async def fetch_user(user_id):
    result = await graph_client.users.by_user_id(user_id).get()
    print(result)
    print(result.display_name)
    print(result.mail)

# 非同期関数を実行
await fetch_user(user_id)

## ユーザー一覧の取得  
会議室もユーザーと同様に取得可能  
https://stackoverflow.com/questions/56288573/isresourceaccount-always-null  
`Is Resource Account` はまだ使えないため、これで判別は難しいかも。後述する会議室一覧取得を使用するといい。  
https://learn.microsoft.com/en-us/graph/api/resources/user?view=graph-rest-1.0#properties

In [ ]:
import asyncio

# ユーザー一覧を取得する非同期関数
async def fetch_users():
    users_response = await graph_client.users.get()  # UserCollectionResponse オブジェクトを取得
    users = users_response.value  # ユーザーリストは value プロパティに格納されている
    for user in users:
        print(f"Email: {user.mail}, ID: {user.id}, Display Name: {user.display_name}, Is Resource Account: {user.is_resource_account}")
        #print(user)

# 非同期関数を実行
await fetch_users()

## カレンダー取得

In [ ]:
import asyncio

# 特定のユーザーのカレンダーを取得する非同期関数
async def fetch_user_calendars(user_id):
    # by_user_id メソッドを使用して特定のユーザーを指定
    user_request = graph_client.users.by_user_id(user_id)
    calendars_response = await user_request.calendars.get()  # User のカレンダーを取得
    calendars = calendars_response.value  # カレンダーリストは value プロパティに格納されている
    for calendar in calendars:
        print(f"ID: {calendar.id}, Name: {calendar.name}")

# 非同期関数を実行
# 取得したいユーザーの ID を指定
await fetch_user_calendars(user_id)

## イベント取得  
https://learn.microsoft.com/ja-jp/graph/api/user-list-events?view=graph-rest-1.0&tabs=python#request

In [ ]:
import asyncio

async def get_user_events(user_id):
    #from msgraph.generated.users.item.events.events_request_builder import EventsRequestBuilder

    #query_params = EventsRequestBuilder.EventsRequestBuilderGetQueryParameters(
    #    select=["subject", "start", "end"],  # 適切なプロパティを指定
    #)

    # 特定のユーザーのイベントを取得
    user_events = graph_client.users.by_user_id(user_id).events
    result = await user_events.get()
    print(result)

    # subject の一覧を出力
    for event in result.value:
        print(event.subject, event.start, event.end, event.location, event.body)

# 非同期関数を実行
await get_user_events(user_id)

## イベント登録  
`transaction_id`を設定することによって重複登録対策可能 UUID等設定

In [ ]:
import asyncio
from datetime import datetime  # datetime モジュールをインポート

async def post_user_events(user_id):
    from msgraph.generated.users.item.events.events_request_builder import EventsRequestBuilder
    from kiota_abstractions.base_request_configuration import RequestConfiguration
    from msgraph.generated.models.event import Event
    from msgraph.generated.models.item_body import ItemBody
    from msgraph.generated.models.body_type import BodyType
    from msgraph.generated.models.date_time_time_zone import DateTimeTimeZone
    from msgraph.generated.models.location import Location
    from msgraph.generated.models.attendee import Attendee
    from msgraph.generated.models.email_address import EmailAddress
    from msgraph.generated.models.attendee_type import AttendeeType

    # 現在の日時を取得してフォーマット
    current_time = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

    request_body = Event(
        subject = f"{current_time}Let's go for lunch",  # subject に日時を追加
        body = ItemBody(
            content_type = BodyType.Html,
            content = "Does noon work for you?",
        ),
        start = DateTimeTimeZone(
            date_time = "2025-04-15T17:00:00",
            time_zone = "Pacific Standard Time",
        ),
        end = DateTimeTimeZone(
            date_time = "2025-04-15T19:00:00",
            time_zone = "Pacific Standard Time",
        ),
        location = Location(
            display_name = "Harry's Bar",
        ),
        allow_new_time_proposals = True,
        # transaction_id = "7E163156-7762-4BEB-A1C6-729EA81755A7",
    )

    request_configuration = RequestConfiguration()
    request_configuration.headers.add("Prefer", "outlook.timezone=\"Pacific Standard Time\"")

    # 特定のユーザーのイベントを登録
    # イベントを作成
    result = await graph_client.users.by_user_id(user_id).events.post(request_body)
    print(result)

# 非同期関数を実行
await post_user_events(user_id)

## 会議室一覧取得  
https://learn.microsoft.com/ja-jp/graph/api/place-list?view=graph-rest-1.0&tabs=python

In [ ]:
import asyncio

async def get_place():

    #result = await graph_client.places.graph_room_list.get()
    result = await graph_client.places.graph_room.get()
    print(result)
    for place in result.value:
        print(f"ID: {place.id}, Display Name: {place.display_name}, Email: {place.email_address}")

# 非同期関数を実行
await get_place()

## 会議室イベント取得

In [ ]:
import asyncio

async def get_user_events(user_id):
    # from msgraph import GraphServiceClient
    # To initialize your graph_client, see https://learn.microsoft.com/en-us/graph/sdks/create-client?from=snippets&tabs=python

    result = await graph_client.users.by_user_id(user_id).calendar.events.get()

    print(result)

    # イベントの詳細を出力
    for event in result.value:
        print(f"件名: {event.subject}")
        print(f"開始日時: {event.start.date_time} ({event.start.time_zone})")
        print(f"終了日時: {event.end.date_time} ({event.end.time_zone})")
        print(f"場所: {event.location.display_name}")
        print(f"参加者:")
        for attendee in event.attendees:
            print(f"  - {attendee.email_address.name} ({attendee.email_address.address})")
        print(f"重要度: {event.importance}")
        print(f"キャンセル済み: {event.is_cancelled}")
        print(f"詳細: {event.body.content}")
        print("-" * 40)

# 非同期関数を実行
# 取得したい会議室の ID を指定
await get_user_events(room_id)

## 空き時間スケジュールを取得 
複数ユーザー、複数会議室指定  
https://learn.microsoft.com/ja-jp/graph/api/calendar-getschedule?view=graph-rest-1.0&tabs=python  

freeBusyStatus	出席者の空き時間の状態。 
https://learn.microsoft.com/ja-jp/graph/api/resources/attendeeavailability?view=graph-rest-1.0#properties  
使用可能な値: free、tentative、busy、oof、workingElsewhere、unknown。  
- free 空き時間
- tentative 仮の予定
- busy 予定あり
- oof 不在
- workingElsewhere 他の場所で仕事中
- unknown


In [ ]:
import asyncio

# メールアドレスを格納するリスト
email_addresses = []
# ユーザー取得の最大件数を指定する変数
max_users = 5  # 必要に応じて変更可能

# .env で指定した user_id のメールアドレスを取得
async def fetch_user(user_id):
    result = await graph_client.users.by_user_id(user_id).get()
    email_addresses.append(result.mail)
    print(result.mail)

# 会議室のメールアドレスを全件取得
async def get_place():
    result = await graph_client.places.graph_room.get()
    for place in result.value:
        email_addresses.append(place.email_address)
        print(place.email_address)

# ユーザー一覧を取得
async def fetch_users():
    users_response = await graph_client.users.get()  # UserCollectionResponse オブジェクトを取得
    users = users_response.value  # ユーザーリストは value プロパティに格納されている
    for user in users[:max_users]:  # 最初のmax_users件のみ取得
        email_addresses.append(user.mail)
        print(user.mail)

# スケジュールを取得
async def get_user_events(user_id):
    from msgraph.generated.users.item.calendar.get_schedule.get_schedule_post_request_body import GetSchedulePostRequestBody
    from msgraph.generated.models.date_time_time_zone import DateTimeTimeZone
    from kiota_abstractions.base_request_configuration import RequestConfiguration

    # スケジュールリクエストボディを作成
    request_body = GetSchedulePostRequestBody(
        schedules=email_addresses,  # 取得したメールアドレスを設定
        start_time=DateTimeTimeZone(
            date_time="2025-03-18T09:00:00",
            time_zone="Asia/Tokyo",
        ),
        end_time=DateTimeTimeZone(
            date_time="2025-04-30T18:00:00",
            time_zone="Asia/Tokyo",
        ),
        availability_view_interval=60,
    )

    request_configuration = RequestConfiguration()
    request_configuration.headers.add("Prefer", "outlook.timezone=\"Asia/Tokyo\"")

    result = await graph_client.users.by_user_id(user_id).calendar.get_schedule.post(
        request_body, request_configuration=request_configuration
    )

    # 結果を整形して表示
    print("スケジュール情報の取得結果:\n")
    for schedule in result.value:
        print(f"メールアドレス: {schedule.schedule_id}")
        if schedule.error:
            print(f"  - エラー: {schedule.error.message}")
            print(f"  - レスポンスコード: {schedule.error.response_code}")
        else:
            print("  - スケジュール情報:")
            if schedule.schedule_items:
                for item in schedule.schedule_items:
                    print(f"    - 開始: {item.start.date_time} ({item.start.time_zone})")
                    print(f"      終了: {item.end.date_time} ({item.end.time_zone})")
                    print(f"      状態: {item.status}")
            else:
                print("    スケジュール情報はありません。")
        print("\n")

# 非同期関数を実行
# 取得したいユーザーの ID を指定
await fetch_user(user_id)  # ユーザーのメールアドレスを取得
await get_place()          # 会議室のメールアドレスを取得
await fetch_users()        # ユーザー一覧からメールアドレスを取得
await get_user_events(user_id)  # スケジュールを取得